In [1]:
import torch
import torch.nn as nn
from torchvision import transforms, datasets, utils
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
import torch.optim as optim
from models.super_efficientnet import Super_EfficientNet, base_model, phi_values
from generate_filter import Kernel
import os
import json
import time
#import cv2

In [2]:
#device : GPU or CPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [3]:
version = "b0"
phi, res, drop_rate = phi_values[version]

In [4]:
data_transform = {
    "train": transforms.Compose([transforms.RandomResizedCrop(res),
                                 transforms.RandomHorizontalFlip(),
                                 transforms.ToTensor(),
                                 transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]),
    "val": transforms.Compose([transforms.Resize((res, res)),  # cannot 224, must (224, 224)
                               transforms.ToTensor(),
                               transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])}

In [5]:
#data_root = os.path.abspath(os.path.join(os.getcwd(), "../.."))  # get data root path
data_root = os.getcwd()
image_path = data_root + "/data/"  # flower data set path
train_dataset = datasets.ImageFolder(root=image_path + "/train",
                                     transform=data_transform["train"])
train_num = len(train_dataset)

In [6]:
cd_list = train_dataset.class_to_idx
cla_dict = dict((val, key) for key, val in cd_list.items())
# write dict into json file
json_str = json.dumps(cla_dict, indent=4)
with open('class_indices_efficient.json', 'w') as json_file:
    json_file.write(json_str)

In [7]:
batch_size = 8
train_loader = torch.utils.data.DataLoader(train_dataset,
                                           batch_size=batch_size, shuffle=True,
                                           num_workers=0)

validate_dataset = datasets.ImageFolder(root=image_path + "/val",
                                        transform=data_transform["val"])
val_num = len(validate_dataset)
validate_loader = torch.utils.data.DataLoader(validate_dataset,
                                              batch_size=batch_size, shuffle=True,
                                              num_workers=0)

test_data_iter = iter(validate_loader)
test_image, test_label = test_data_iter.next()
print(test_image[0].size(),type(test_image[0]))
print(test_label[0],test_label[0].item(),type(test_label[0]))

torch.Size([3, 224, 224]) <class 'torch.Tensor'>
tensor(8) 8 <class 'torch.Tensor'>


In [8]:
kernels = Kernel()().to(device)

net = Super_EfficientNet(filters=kernels, version=version,num_classes=9)

net.to(device)

loss_function = nn.CrossEntropyLoss()

optimizer = optim.Adam(net.parameters(), lr=0.0002)

save_path = './Super_EfficientNet.pth'

best_acc = 0.0

In [9]:
for epoch in range(50):
    # train
    net.train()
    running_loss = 0.0
    t1 = time.perf_counter()
    for step, data in enumerate(train_loader, start=0):
        images, labels = data
        #print(labels.shape)
        optimizer.zero_grad()
        outputs = net(images.to(device))
        loss = loss_function(outputs, labels.to(device))
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        # print train process
        rate = (step + 1) / len(train_loader)
        a = "*" * int(rate * 50)
        b = "." * int((1 - rate) * 50)
        print("\rtrain loss: {:^3.0f}%[{}->{}]{:.3f}".format(int(rate * 100), a, b, loss), end="")
    print()
    print(time.perf_counter()-t1)

    # validate
    net.eval()  
    acc = 0.0  # accumulate accurate number / epoch
    with torch.no_grad():
        for val_data in validate_loader:
            val_images, val_labels = val_data
            outputs = net(val_images.to(device))
            predict_y = torch.max(outputs, dim=1)[1]
            acc += (predict_y == val_labels.to(device)).sum().item()
        val_accurate = acc / val_num
        if val_accurate > best_acc:
            best_acc = val_accurate
            torch.save(net.state_dict(), save_path)
        print('[epoch %d] train_loss: %.3f  test_accuracy: %.3f' %
              (epoch + 1, running_loss / step, val_accurate))

print('Finished Training')

train loss: 100%[**************************************************->]2.207
107.3285228
[epoch 1] train_loss: 2.017  test_accuracy: 0.328
train loss: 100%[**************************************************->]1.613
109.7728971
[epoch 2] train_loss: 1.833  test_accuracy: 0.396
train loss: 100%[**************************************************->]1.737
111.1802318
[epoch 3] train_loss: 1.733  test_accuracy: 0.469
train loss: 100%[**************************************************->]1.754
110.55919059999997
[epoch 4] train_loss: 1.620  test_accuracy: 0.493
train loss: 100%[**************************************************->]1.749
110.59709739999994
[epoch 5] train_loss: 1.540  test_accuracy: 0.530
train loss: 100%[**************************************************->]1.905
200.85295280000003
[epoch 6] train_loss: 1.490  test_accuracy: 0.552
train loss: 100%[**************************************************->]1.786
111.28914369999995
[epoch 7] train_loss: 1.445  test_accuracy: 0.556
train 

In [12]:
model = Super_EfficientNet(filters=kernels,num_classes=9, version=version)
save_path = "Super_EfficientNet.pth"
model.load_state_dict(torch.load(save_path))
model.eval()

Super_EfficientNet(
  (pool): AdaptiveAvgPool2d(output_size=1)
  (features): Sequential(
    (0): CNNBlock(
      (cnn): Conv2d(504, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (silu): SiLU()
    )
    (1): InvertedResidualBlock(
      (conv): Sequential(
        (0): CNNBlock(
          (cnn): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (silu): SiLU()
        )
        (1): SqueezeExcitation(
          (se): Sequential(
            (0): AdaptiveAvgPool2d(output_size=1)
            (1): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
            (2): SiLU()
            (3): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
            (4): Sigmoid()
          )
        )
        (2): Conv2d(32, 16, kernel_size=(1, 1), 